In [25]:
import sys
sys.path.append('/afs/ee.cooper.edu/user/t/a/tam8/documents/ml_misc/semi_supervised/')
import coreg
reload(coreg)
import trireg
reload(trireg)
sys.path.append('/afs/ee.cooper.edu/user/t/a/tam8/documents/ml_misc/ordinal/')
import simple
reload(simple)

import pandas as pd
import numpy as np
from sklearn import preprocessing
import xgboost as xgb

from transformers import *
from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion, make_union

import metrics
reload(metrics)
from sklearn.cross_validation import StratifiedKFold

from sklearn.base import clone

from sklearn.svm import SVC, LinearSVC, SVR
from sklearn.linear_model import LinearRegression, LogisticRegression, ElasticNet, Ridge, Lasso, SGDRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, ExtraTreesClassifier, ExtraTreesRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.isotonic import IsotonicRegression


import minirank as mr

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from time import time
from collections import Counter

In [2]:
# Load Data
import cPickle as pickle
train_pd  = pd.read_pickle('saved/train_pd_enc.p')
test_pd  = pd.read_pickle('saved/test_pd_enc.p')
labels = pd.read_pickle('saved/labels.p')
test_ind = pickle.load(open('saved/test_ind.p'))



In [4]:
train = np.array(train_pd)
test = np.array(test_pd)

X_train = train.astype(float)
X_test = test.astype(float)
y_train = np.array(labels)

pipe_x = make_pipeline(
    make_union(
        IdentityTformer(),
#         make_pipeline(AddTformer(1), BoxCoxTformer()),
#         AnscombeTformer(),
    ),
#     StandardScaler(),
)
pipe_y = make_pipeline(
    IdentityTformer(),
#     tforms.BoxCoxTformer(),
#     tforms.LogTfortforms.mer(),
#     tforms.AnscombeTformer(),
#     tforms.FreemanTukeyTformer(),
#     tforms.ArcsinhTformer(),
#     StandardScaler(),
    
)
pipe_x.fit(np.r_[X_train, X_test])


X_train = pipe_x.transform(X_train)
X_test = pipe_x.transform(X_test)
y_train = pipe_y.fit_transform(y_train)

# # small_n = 5000
# X_train = X_train[:small_n,:]
# y_train = y_train[:small_n]

print y_train.shape
print X_train.shape
print X_test.shape
print len(np.unique(y_train))

(50999,)
(50999, 111)
(51000, 111)
50


# XGB

In [28]:
print y_train
y_bin = (y_train > 50).astype(int)
print Counter(y_bin)


skf = StratifiedKFold(y=y_bin, n_folds=5)
for ind_train, ind_val in skf:
    X_train_k, y_train_bin_k = X_train[ind_train], y_bin[ind_train]
    X_val_k, y_val_bin_k = X_train[ind_val], y_bin[ind_val]
    break

[ 1  4  1 ...,  3 14  9]
Counter({0: 50994, 1: 5})


In [29]:
params = {}
params["objective"] = "binary:logistic"
# params["objective"] = "multi:softprob"
params["eta"] = 0.01
params["min_child_weight"] = 5
params["subsample"] = 0.8
params["colsample_bytree"] = 0.8
# params["max_delta_step"] = 10
params["silent"] = 1
params["max_depth"] = 8 #7
# params["scale_pos_weight"] = 1.0

params["eval_metric"] = "logloss"
params["gamma"] = 0.5


plst = list(params.items())

max_rounds = 1000
xgtest = xgb.DMatrix(X_test)

#create a train and validation dmatrices 
xgtrain = xgb.DMatrix(X_train_k, label=y_train_bin_k)
xgval = xgb.DMatrix(X_val_k, label=y_val_bin_k)

#train using early stopping and predict
watchlist = [(xgtrain, 'train'),(xgval, 'val')]
model = xgb.train(plst, xgtrain, max_rounds, watchlist, early_stopping_rounds=20)
# preds1 = model.predict(xgtest)


Will train until val error hasn't decreased in 20 rounds.
[0]	train-logloss:0.683175	val-logloss:0.683212
[1]	train-logloss:0.673416	val-logloss:0.673462
[2]	train-logloss:0.663881	val-logloss:0.663890
[3]	train-logloss:0.654545	val-logloss:0.654534
[4]	train-logloss:0.645330	val-logloss:0.645334
[5]	train-logloss:0.636303	val-logloss:0.636317
[6]	train-logloss:0.627459	val-logloss:0.627470
[7]	train-logloss:0.618833	val-logloss:0.618791
[8]	train-logloss:0.610293	val-logloss:0.610266
[9]	train-logloss:0.601891	val-logloss:0.601906
[10]	train-logloss:0.593708	val-logloss:0.593706
[11]	train-logloss:0.585652	val-logloss:0.585649
[12]	train-logloss:0.577724	val-logloss:0.577722
[13]	train-logloss:0.569931	val-logloss:0.569952
[14]	train-logloss:0.562289	val-logloss:0.562304
[15]	train-logloss:0.554788	val-logloss:0.554799
[16]	train-logloss:0.547411	val-logloss:0.547434
[17]	train-logloss:0.540161	val-logloss:0.540189
[18]	train-logloss:0.533063	val-logloss:0.533056
[19]	train-logloss:0.

# Generate submission

In [9]:
preds = pd.DataFrame({"Id": test_ind, "Hazard": preds})
preds = preds.set_index('Id')
preds.to_csv('xgboost_benchmark.csv')

array([[ 15.,   3.,   2., ...,   4.,   2.,   2.],
       [ 16.,  14.,   5., ...,   4.,   2.,   1.],
       [ 10.,  10.,   5., ...,   4.,   6.,   1.],
       ..., 
       [ 18.,   7.,   5., ...,   4.,   1.,   1.],
       [ 18.,  17.,   5., ...,   2.,   2.,   6.],
       [  5.,  15.,   3., ...,   4.,   5.,   4.]])